[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/giswqs/notebook-share/blob/master/docs/pakistan_floods.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/giswqs/notebook-share&urlpath=lab/tree/notebook-share/docs/pakistan_floods.ipynb&branch=master)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%pakistan_floods.ipynb)

# Visualization and Analysis of Pakistan Floods 2022 Using Earth Engine and Geemap



## Introduction

From 14 June to October 2022, floods in Pakistan killed 1,739 people, and caused ₨ 3.2 trillion ($14.9 billion) of damage and ₨ 3.3 trillion ($15.2 billion) of economic losses. The flooding was the world's deadliest flood since the 2020 South Asian floods and described as the worst in the country's history. On 25 August, Pakistan declared a state of emergency because of the flooding.

**Sources:**
- Wikipedia: [2022 Pakistan floods](https://en.wikipedia.org/wiki/2022_Pakistan_floods)
- UNICEF: [Devastating floods in Pakistan](https://www.unicef.org/emergencies/devastating-floods-pakistan-2022)


## Installation

In [1]:
# !pip install geemap

## Import libraries

In [2]:
import ee
import geemap.foliumap as geemap

## Create an interactive map

In [3]:
Map = geemap.Map()

country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', 'Pakistan'))

style = {'color': 'red', 'fillColor': '00000000'}
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map.centerObject(country)

Map

## Create Landsat composites

In [4]:
Map = geemap.Map()

col_2021 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
    .filterDate('2021-06-01', '2021-10-31') \
    .filterBounds(country) \

landsat_2021 = ee.Algorithms.Landsat.simpleComposite(col_2021) \
    .clipToCollection(country)

vis_params = {'bands': ['B6', 'B5', 'B4'], 'max': 128}
Map.addLayer(landsat_2021, vis_params, 'Landsat 2021')


In [5]:
col_2022 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
    .filterDate('2022-06-01', '2022-10-31') \
    .filterBounds(country) \

landsat_2022 = ee.Algorithms.Landsat.simpleComposite(col_2022) \
    .clipToCollection(country)

Map.addLayer(landsat_2022, vis_params, 'Landsat 2022')
Map.centerObject(country)
Map

## Compare Landsat composites side by side

In [6]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(landsat_2021, vis_params, 'Landsat 2021')
right_layer = geemap.ee_tile_layer(landsat_2022, vis_params, 'Landsat 2022')

Map.split_map(left_layer, right_layer, left_label='Landsat 2021', right_label='Landsat 2022')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Compute Normalized Difference Water Index (NDWI)

In [7]:
ndwi_2021 = landsat_2021.normalizedDifference(['B3', 'B5']).rename('NDWI')
ndwi_2022 = landsat_2022.normalizedDifference(['B3', 'B5']).rename('NDWI')

In [8]:
ndwi_vis = {'min': -1, 'max': 1, 'palette': 'ndwi'}

In [9]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

left_layer = geemap.ee_tile_layer(ndwi_2021, ndwi_vis, 'NDWI 2021')
right_layer = geemap.ee_tile_layer(ndwi_2022, ndwi_vis, 'NDWI 2022')

Map.split_map(left_layer, right_layer, left_label='NDWI 2021', right_label='NDWI 2022')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map


## Extract water extent

In [10]:
threshold = 0
water_2021 = ndwi_2021.gt(threshold)
water_2022 = ndwi_2022.gt(threshold)

In [11]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021')
right_layer = geemap.ee_tile_layer(water_2022.selfMask(), {'palette': 'red'}, 'Water 2022')

Map.split_map(left_layer, right_layer, left_label='Water 2021', right_label='Water 2022')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map


## Extract flood extent

In [12]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

In [13]:
Map = geemap.Map()
Map.setCenter(68.4338, 26.4213, 7)

Map.addLayer(landsat_2021, vis_params, 'Landsat 2021', False)
Map.addLayer(landsat_2022, vis_params, 'Landsat 2022', False)

left_layer = geemap.ee_tile_layer(water_2021.selfMask(), {'palette': 'blue'}, 'Water 2021')
right_layer = geemap.ee_tile_layer(water_2022.selfMask(), {'palette': 'red'}, 'Water 2022')

Map.split_map(left_layer, right_layer, left_label='Water 2021', right_label='Water 2022')

Map.addLayer(flood_extent, {'palette': 'cyan'}, 'Flood Extent')
Map.addLayer(country.style(**style), {}, 'Pakistan')
Map

## Calculate flood area

In [14]:
area_2021 = geemap.image_area_by_group(water_2021.selfMask(), scale=1000, denominator=1e6, decimal_places=4)
area_2021

Calculating area for group 1 ...


,area,percentage
group,,
1,29543.0874,1.0


In [15]:
area_2022 = geemap.image_area_by_group(water_2022.selfMask(), scale=1000, denominator=1e6, decimal_places=4)
area_2022

Calculating area for group 1 ...


,area,percentage
group,,
1,34828.4376,1.0


In [16]:
flood_area = geemap.image_area_by_group(flood_extent.selfMask(), scale=1000, denominator=1e6, decimal_places=4)
flood_area

Calculating area for group 1 ...


,area,percentage
group,,
1,12256.3199,1.0
